# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [45]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key



### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [46]:
output_data = "../Python_Challenges_WeatherPy/Output/weather_data.csv"
# output_data_file

data = pd.read_csv(output_data)

weather_data = data.drop(columns=["Unnamed: 0"])
weather_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,jamestown,42.10,-79.24,48.99,100,90,5.82,US,1603248192
1,new norfolk,6.65,-10.59,72.59,98,65,1.61,LR,1603248194
2,buarcos,40.17,-8.88,60.01,93,40,5.82,PT,1603248195
3,port blair,-7.03,29.76,72.84,73,24,5.21,CD,1603248028
4,christchurch,-43.53,172.63,64.00,55,17,10.29,NZ,1603248017
...,...,...,...,...,...,...,...,...,...
549,marsa matruh,35.88,14.50,63.00,82,0,5.82,MT,1603248382
550,ilhabela,-23.78,-45.36,71.60,88,29,1.14,BR,1603248382
551,tessalit,20.20,1.01,79.11,17,0,9.95,ML,1603248260
552,kintampo,8.06,-1.73,71.62,94,86,3.98,GH,1603248344


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [47]:
gmaps.configure(api_key=g_key)

In [48]:
locations = weather_data[["Lat", "Lng"]]
# locations

humidity = weather_data["Humidity"].astype(float)
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [49]:
new_data = pd.DataFrame(weather_data, columns = ["City", "Max Temp", "Wind Speed", "Cloudiness"])
# A max temperature lower than 80 degrees but higher than 70.
new_max_temp = (new_data["Max Temp"] <= 80) & (new_data["Max Temp"] > 70)
# Wind speed less than 10 mph.
new_wind_speed = new_data["Wind Speed"] < 10
# Zero cloudiness.
new_cloudiness = new_data["Cloudiness"] == 0

# Drop any rows that don't contain all three conditions. 
new_weather_data =new_data[new_max_temp & new_wind_speed & new_cloudiness]

new_weather_data_merged=pd.merge(new_weather_data, weather_data, on=["City","Max Temp","Wind Speed","Cloudiness"], how="left")
new_weather_data_merged

,City,Max Temp,Wind Speed,Cloudiness,Lat,Lng,Humidity,Country,Date
0,villarrica,74.73,3.20,0,-25.75,-56.43,65,PY,1603248039
1,san miguel,79.86,7.45,0,13.30,-4.90,51,ML,1603248039
2,mecca,75.45,6.22,0,21.43,39.83,18,SA,1603247972
3,bandarbeyla,78.67,8.90,0,9.49,50.81,68,SO,1603247874
4,yichang,72.00,4.72,0,30.71,111.28,39,CN,1603248062
5,avenal,75.00,1.01,0,36.00,-120.13,38,US,1603248130
6,san patricio,79.86,7.45,0,13.30,-4.90,51,ML,1603248039
7,bilma,76.10,3.31,0,18.69,12.92,18,NE,1603248145
8,san carlos de bariloche,79.86,7.45,0,13.30,-4.90,51,ML,1603248152
9,sao joao da barra,77.59,3.13,0,20.51,76.21,75,IN,1603248081


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [55]:
hotel_df = pd.DataFrame(new_weather_data_merged, columns=["City", "Country", "Lat", "Lng"])

hotel_df["Hotel Name"] = ""


hotel_df

,City,Country,Lat,Lng,Hotel Name
0,villarrica,PY,-25.75,-56.43,
1,san miguel,ML,13.30,-4.90,
2,mecca,SA,21.43,39.83,
3,bandarbeyla,SO,9.49,50.81,
4,yichang,CN,30.71,111.28,
5,avenal,US,36.00,-120.13,
6,san patricio,ML,13.30,-4.90,
7,bilma,NE,18.69,12.92,
8,san carlos de bariloche,ML,13.30,-4.90,
9,sao joao da barra,IN,20.51,76.21,


In [56]:
hotel_name = []
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

# taken from Airpost_Ratings Example done in class
print(f"----------------------\nRetrieving Results\n----------------------")
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    city = row['City']
    lat = row["Lat"]
    lng = row["Lng"]
    # change location each iteration while leaving original params in place
    
    params["location"] = f"{lat},{lng}"

    # make request
    response = requests.get(base_url, params=params).json()
    try:
        print(f"The first hotel in {city} is {response['results'][0]['name']}.")
        hotel_name.append(response['results'][0]['name'])
    except IndexError:
        print(f"----------------------Missing field/result... skipping.----------------------")
        hotel_name.append(np.nan)
print(f"----------------------\nRetrieving Complete\n----------------------")

----------------------
Retrieving Results
----------------------
The first hotel in villarrica is Villarrica Palace Hotel.
The first hotel in san miguel is Hotel Santoro.
The first hotel in mecca is Pullman Zamzam Makkah.
The first hotel in bandarbeyla is Super secret pirate cove.
The first hotel in yichang is Crowne Plaza Yichang.
The first hotel in avenal is Lodge House Inn The.
The first hotel in san patricio is Hotel Santoro.
----------------------Missing field/result... skipping.----------------------
The first hotel in san carlos de bariloche is Hotel Santoro.
The first hotel in sao joao da barra is Hotel Sai Residency.
The first hotel in san quintin is Hotel Santoro.
The first hotel in aswan is Sofitel Legend Old Cataract Aswan.
The first hotel in sao filipe is Hotel Sai Residency.
The first hotel in dubai is Sheraton Grand Hotel, Dubai.
The first hotel in alexandria is Four Seasons Hotel Alexandria at San Stefano, Egypt.
The first hotel in san juan de colon is Hotel Santoro.
Th

In [57]:
hotel_df["Hotel Name"]=hotel_name
reduced_hotel_df=hotel_df.head(10)
reduced_hotel_df

,City,Country,Lat,Lng,Hotel Name
0,villarrica,PY,-25.75,-56.43,Villarrica Palace Hotel
1,san miguel,ML,13.30,-4.90,Hotel Santoro
2,mecca,SA,21.43,39.83,Pullman Zamzam Makkah
3,bandarbeyla,SO,9.49,50.81,Super secret pirate cove
4,yichang,CN,30.71,111.28,Crowne Plaza Yichang
5,avenal,US,36.00,-120.13,Lodge House Inn The
6,san patricio,ML,13.30,-4.90,Hotel Santoro
7,bilma,NE,18.69,12.92,NaN
8,san carlos de bariloche,ML,13.30,-4.90,Hotel Santoro
9,sao joao da barra,IN,20.51,76.21,Hotel Sai Residency


In [58]:
# NOTE: Do not change any of the code in this cell
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in reduced_hotel_df.iterrows()]
locations = reduced_hotel_df[["Lat", "Lng"]]

In [59]:


# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(markers)
fig.add_layer(heat_layer)


# Display figure


fig



Figure(layout=FigureLayout(height='420px'))

In [63]:
# Output final csv
final_data= pd.merge(hotel_df,new_weather_data_merged,on=["City","Country","Lat","Lng"],how="left")
fina_data_dropna=final_data.dropna()
output_data = "Output/final_data.csv"
fina_data_dropna.to_csv(output_data)

